In [1]:
from nilearn import datasets
import pickle
import json
import numpy as np
import networkx as nx
# from scipy import stats
from statsmodels.stats import nonparametric
from statsmodels.tsa import stattools
from multiprocessing import Pool
import os
from esfMRI import stats_tests, plot_graph_measures, get_periods
from graph import multi_process

In [2]:
filename = "60"
os.makedirs(f"graph_theory/{filename}", exist_ok=True)
atlas_labels = datasets.fetch_atlas_aal()['labels']
threshold = 50

In [ ]:
with open(f"dFC/{filename}dFCs.pkl", "rb") as f:
    dFCs = pickle.load(f)

In [ ]:
save_path = f"graph_theory/dFC/{filename}"
os.makedirs(save_path, exist_ok=True)
with Pool(5) as pool:
    tasks = pool.starmap(multi_process, [[dFCs[sub], sub, threshold, save_path] for sub in dFCs])
graphs = {sub:graph for graph, sub in tasks}
print("全部计算完成，正在保存...")
with open(f"graph_theory/dFC/{filename}_graphs.pkl", "wb") as f:
    pickle.dump(graphs, f)
print("保存完成")

全部计算完成，正在保存...
保存完成


In [2]:
filename = 60
with open(f"graph_theory/dFC/{filename}_graphs.pkl", "rb") as f:
# with open(f"graph_theory/dFC/sub-292.pkl", "rb") as f:
    graphs = pickle.load(f)

In [3]:
keys = ['k_core', 'clustering', 'modularity', 'degree_assortativity_coefficient', 'global_efficiency', 'local_efficiency', 'ratio']

In [ ]:
# 对每个参数、每个被试应用统计学分析
stationary = {}
adfs = {}
kpsss = {}
unknown = []
for key in keys:
    stationary[key] = {}
    adfs[key] = {}
    kpsss[key] = {}
    for sub in graphs:
        stationary[key][sub] = {"ses-preop":{}, "ses-postop":{}}
        adfs[key][sub] = {"ses-preop":{}, "ses-postop":{}}
        kpsss[key][sub] = {"ses-preop":{}, "ses-postop":{}}
        for run in graphs[sub]["ses-preop"]:
            measures = []
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures.append(graph[key])
            try:
                # ADF检验。H0：存在单位根，序列不平稳；H1：不存在单位根，序列平稳
                adf, p_value, usedlag, nobs, critical_values, icbest = stattools.adfuller(measures)
                adfs[key][sub]["ses-preop"][run] = p_value
                # KPSS检验。H0：序列平稳；H1：序列不平稳
                kpss, kp_value, lags, crit = stattools.kpss(measures)
                kpsss[key][sub]["ses-preop"][run] = kp_value
                if p_value < 0.05 and kp_value > 0.05:
                    stationary[key][sub]["ses-preop"][run] = True
                elif p_value > 0.05 and kp_value < 0.05:
                    stationary[key][sub]["ses-preop"][run] = False
                else:
                    unknown.append([sub, "ses-preop", run])
            except Exception as e:
                # print(e)
                stationary[key][sub]["ses-preop"][run] = None
        for run in graphs[sub]["ses-postop"]:
            measures = []
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures.append(graph[key])
            try:
                adf, pvalue, usedlag, nobs, critical_values, icbest = stattools.adfuller(measures)
                adfs[key][sub]["ses-postop"][run] = p_value
                kpss, kp_value, lags, crit = stattools.kpss(measures)
                kpsss[key][sub]["ses-postop"][run] = kp_value
                if p_value < 0.05 and kp_value > 0.05:
                    stationary[key][sub]["ses-postop"][run] = True
                elif p_value > 0.05 and kp_value < 0.05:
                    stationary[key][sub]["ses-postop"][run] = False
                else:
                    unknown.append([sub, "ses-postop", run])
            except Exception as e:
                # print(e)
                stationary[key][sub]["ses-postop"][run] = None
with open("stationary.json", "w") as f:
    json.dump(stationary, f)

In [4]:
# 对每个参数、每个被试应用统计学分析
no_changed = {}
change = {}
for key in keys:
    no_changed[key] = []
    change[key] = {}
    for sub in graphs:
        measures_preop = []
        measures_postop = []
        for run in graphs[sub]["ses-preop"]:
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop.append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop.append(graph[key])
        p_values = stats_tests(measures_preop, measures_postop, False)
        if np.any(p_values[4:5] > 0.05):
            no_changed[key].append(sub)
        else:
            change[key][sub] = {}
            change[key][sub]["rank_cmp"] = nonparametric.rank_compare_2indep(measures_preop, measures_postop).tuple
            change[key][sub]["change"] = "up" if change[key][sub]["rank_cmp"][0] < 0 else "down"
with open(f"graph_theory/{filename}/no_changed_dFC.json", "w") as f:
    json.dump(no_changed, f, indent=4)
with open(f"graph_theory/{filename}/changed_dFC.json", "w") as f:
    json.dump(change, f, indent=4)

In [10]:
for key in keys:
    cnt_up = 0
    cnt_down = 0
    for sub in change[key]:
        if change[key][sub]["change"] == "up":
            cnt_up += 1
            if key != "modularity" and key != "degree_assortativity_coefficient" and key != "ratio":
                print(sub)
        else:
            cnt_down += 1
    print(key, "\tup:", cnt_up, "down: ", cnt_down)

sub-303
sub-320
sub-334
sub-352
k_core 	up: 4 down:  16
sub-303
sub-320
sub-334
sub-352
clustering 	up: 4 down:  16
modularity 	up: 13 down:  7
degree_assortativity_coefficient 	up: 4 down:  12
sub-303
sub-320
sub-334
sub-352
global_efficiency 	up: 4 down:  16
sub-303
sub-320
sub-334
sub-352
local_efficiency 	up: 4 down:  16
ratio 	up: 16 down:  2


In [9]:
# 对每个参数整体应用统计学分析
for key in keys:
    measures_preop = []
    measures_postop = []
    for sub in graphs:
        for run in graphs[sub]["ses-preop"]:
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop.append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop.append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop.append(graph[key])
    p_values = stats_tests(measures_preop, measures_postop, False)
    if np.any(p_values[4:5] > 0.05):
        print(key, "无明显变化", p_values)
    else:
        print(key)
        print(nonparametric.rank_compare_2indep(measures_preop, measures_postop).tuple)
    print()

k_core
(49.00559439969875, 0.0)

clustering
(58.3535228339455, 0.0)



c:\Users\12500\MiniConda3\envs\py311\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


modularity
(-23.867663981295447, 2.6560391418747775e-122)

degree_assortativity_coefficient
(7.347009244358477, 2.1358948284837715e-13)

global_efficiency
(58.33093527915202, 0.0)

local_efficiency
(58.41496580458525, 0.0)

ratio
(-32.739858428418415, 9.371298949000868e-227)



In [5]:
# 绘制参数变化
for key in keys:
    save_path = f"graph_theory/{filename}/{key}"
    os.makedirs(save_path, exist_ok=True)
    for sub in graphs:
        measures_preop = {}
        measures_postop = {}
        for run in graphs[sub]["ses-preop"]:
            measures_preop[run] = []
            for graph in graphs[sub]["ses-preop"][run]:
                if key == "clustering":
                    measures_preop[run].append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_preop[run].append(graph[key])
        for run in graphs[sub]["ses-postop"]:
            measures_postop[run] = []
            for graph in graphs[sub]["ses-postop"][run]:
                if key == "clustering":
                    measures_postop[run].append(sum(graph[key].values())/116)
                elif not np.isnan(graph[key]):
                    measures_postop[run].append(graph[key])
        plot_graph_measures(measures_preop, measures_postop, key, f"{save_path}/{sub}.png")